In [1]:
"""Imports"""
import numpy as np
import pandas as pd

from pathlib import Path

After looking at the data in a text editor, it is clear there are extrenous rows that can be removed.  Further, for the Benzene portion, we only need select columns

In [2]:
"""Load up the raw data, converting European data format to US standards"""
path_to_file = Path(Path.cwd().parents[1], "Data", "AirQualityUCI.csv")
raw = pd.read_csv(path_to_file, delimiter=";", decimal=",")
raw_benzene = (
    raw
    .filter(["Date", "Time", "C6H6(GT)", "PT08.S5(O3)", "T", "RH", "AH"])
    .rename(columns={"T" : "Temperature (C)",
                    "RH" : "Relative Humidity",
                    "AH" : "Absolute Humidity"})
).iloc[:9357]

raw_benzene.head()

,Date,Time,C6H6(GT),PT08.S5(O3),Temperature (C),Relative Humidity,Absolute Humidity
0,10/03/2004,18.00.00,11.9,1268.0,13.6,48.9,0.7578
1,10/03/2004,19.00.00,9.4,972.0,13.3,47.7,0.7255
2,10/03/2004,20.00.00,9.0,1074.0,11.9,54.0,0.7502
3,10/03/2004,21.00.00,9.2,1203.0,11.0,60.0,0.7867
4,10/03/2004,22.00.00,6.5,1110.0,11.2,59.6,0.7888


It appears that the scale is different between the Ground Truth and the new sensor measurements.  We'll need to normalize them.

In [3]:
"""Check out statistics abou"""
raw_benzene.describe()

,C6H6(GT),PT08.S5(O3),Temperature (C),Relative Humidity,Absolute Humidity
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,1.865683,975.072032,9.778305,39.485380,-6.837604
std,41.380206,456.938184,43.203623,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,4.000000,700.000000,10.900000,34.100000,0.692300
50%,7.900000,942.000000,17.200000,48.600000,0.976800
75%,13.600000,1255.000000,24.100000,61.900000,1.296200
max,63.700000,2523.000000,44.600000,88.700000,2.231000


We know that `-200` means that the value is missing.

In [4]:
"""Take a look at how many values may be missing"""
raw_benzene[raw_benzene["PT08.S5(O3)"] == -200][raw_benzene.columns[2:-1]]

,C6H6(GT),PT08.S5(O3),Temperature (C),Relative Humidity
524,-200.0,-200.0,-200.0,-200.0
525,-200.0,-200.0,-200.0,-200.0
526,-200.0,-200.0,-200.0,-200.0
701,-200.0,-200.0,-200.0,-200.0
702,-200.0,-200.0,-200.0,-200.0
...,...,...,...,...
8111,-200.0,-200.0,-200.0,-200.0
8112,-200.0,-200.0,-200.0,-200.0
8113,-200.0,-200.0,-200.0,-200.0
8114,-200.0,-200.0,-200.0,-200.0


It appears if the dataset is missing a value from our sensor, it is missing a value for every field.  Let's test that theory.

In [5]:
"""Let's see if a missing value from our sensor also means a missing value for everything at that timestamp """
def compare_all_columns(df):
    array = df.to_numpy()
    return (array[0] == array).all()

compare_all_columns(raw_benzene[raw_benzene["C6H6(GT)"] == -200][raw_benzene.columns[2:-1]])

np.True_

We have confirmed this to be the case.  Now, we need to see if there are any missing values from other fields that don't correspond to a missing value with our sensor data.

In [6]:
for column in raw_benzene.columns:
    print(column, (raw_benzene[column] == -200).sum())

Date 0
Time 0
C6H6(GT) 366
PT08.S5(O3) 366
Temperature (C) 366
Relative Humidity 366
Absolute Humidity 366


With the expectation that the date and time fields are part of record keeping rather than transmission, we can see that there are timestamps with no values from any sensor.  This implies network connectivity issues of some sort, likely on the recieving end since there are two different devices sending data through two different network interfaces.

In [7]:
"""Before"""
interpolated = raw_benzene.copy()
interpolated.iloc[523:528]

,Date,Time,C6H6(GT),PT08.S5(O3),Temperature (C),Relative Humidity,Absolute Humidity
523,01/04/2004,13.00.00,8.6,790.0,21.8,33.9,0.8771
524,01/04/2004,14.00.00,-200.0,-200.0,-200.0,-200.0,-200.0000
525,01/04/2004,15.00.00,-200.0,-200.0,-200.0,-200.0,-200.0000
526,01/04/2004,16.00.00,-200.0,-200.0,-200.0,-200.0,-200.0000
527,01/04/2004,17.00.00,16.4,1187.0,23.6,27.5,0.7913


In [8]:
"""Replace missing values using liear interpolation"""
numeric_cols = interpolated.select_dtypes(include=[np.number]).columns

interpolated[numeric_cols] = (
    interpolated[numeric_cols]
    .replace(-200.0, np.nan)
    .interpolate()
)

"""After"""
interpolated.iloc[523:528]

,Date,Time,C6H6(GT),PT08.S5(O3),Temperature (C),Relative Humidity,Absolute Humidity
523,01/04/2004,13.00.00,8.60,790.00,21.80,33.9,0.87710
524,01/04/2004,14.00.00,10.55,889.25,22.25,32.3,0.85565
525,01/04/2004,15.00.00,12.50,988.50,22.70,30.7,0.83420
526,01/04/2004,16.00.00,14.45,1087.75,23.15,29.1,0.81275
527,01/04/2004,17.00.00,16.40,1187.00,23.60,27.5,0.79130


In [9]:
"""Normalize the data in each row"""
normalized = interpolated.copy()
for column in numeric_cols:
	minimum = interpolated[column].min()
	maximum = interpolated[column].max()
	normalized[column] = (interpolated[column] - minimum) / (maximum - minimum)
normalized.head()

,Date,Time,C6H6(GT),PT08.S5(O3),Temperature (C),Relative Humidity,Absolute Humidity
0,10/03/2004,18.00.00,0.185535,0.454822,0.333333,0.499371,0.280066
1,10/03/2004,19.00.00,0.146226,0.326238,0.326882,0.484277,0.264282
2,10/03/2004,20.00.00,0.139937,0.370547,0.296774,0.563522,0.276352
3,10/03/2004,21.00.00,0.143082,0.426586,0.277419,0.638994,0.294190
4,10/03/2004,22.00.00,0.100629,0.386186,0.281720,0.633962,0.295216


As can be seen, our sensor is measuring for `O3` while the ground truth is a measure of `C6H6 (Benzene)`.  The original intention of the dataset must have been to estimate the benzen concetrations based on O3 concentrations, temperature, and humidity.  This will complicate our model and needs to be taken into account.

In [10]:
"""Fix Date and time formatting and data type"""
benzene = interpolated.copy()
benzene["Time"] = benzene["Time"].str.replace(".", ":", regex=False)
benzene["Datetime"] = pd.to_datetime(
    benzene[["Date", "Time"]]
    .apply(lambda x: 
           " ".join(x.values.astype(str)),
           axis="columns"), 
           dayfirst=True)

benzene["Date"] = benzene["Datetime"].dt.date
benzene["Time"] = benzene["Datetime"].dt.time

benzene.dtypes

Date                         object
Time                         object
C6H6(GT)                    float64
PT08.S5(O3)                 float64
Temperature (C)             float64
Relative Humidity           float64
Absolute Humidity           float64
Datetime             datetime64[ns]
dtype: object

In [ ]:
"""Save the cleaned data for furute use"""
path_to_save = Path(Path.cwd().parents[1], "Data", "Benzene.csv")
benzene.to_csv(path_to_save)